In [1]:
import pandas as pd
import json
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, TimeoutException, WebDriverException, ElementClickInterceptedException, InvalidElementStateException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from datetime import datetime
from selenium import webdriver
import time

In [2]:
df_tags=pd.read_excel('tags.xlsx')
df_tags

,id,text
0,1,desenvolvimento de pessoas
1,2,política nacional de desenvolvimento de pessoas
2,3,PNDP
3,4,inovação
4,5,melhoria
5,6,ações implementadas
6,7,publicidade
7,8,capacitação
8,9,cursos
9,10,conhecimento


In [26]:
json_data=df_tags.to_json(orient='records',force_ascii=False)

In [27]:
# Salva o objeto JSON em um arquivo
with open('tags.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

<h1>Baixando tags do portal</h1>

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.81M/6.81M [00:01<00:00, 4.36MB/s]
C:\Users\Alexandre Cunha\AppData\Local\Temp\ipykernel_18572\1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
tags=driver.find_elements(By.CLASS_NAME, 'tags')

In [8]:
lista_tags=[]
for tag in tags:
    links=tag.find_elements(By.CLASS_NAME, 'link-produtos')
    for link_produto in links:
        lista_tags.append(link_produto.text)

In [9]:
len(lista_tags)

421

In [25]:
df_tags=pd.DataFrame()
df_tags['TAGS']=lista_tags
df_tags=df_tags.drop_duplicates()
df_tags=df_tags.reset_index()
df_tags=df_tags.drop('index', axis=1)
df_tags=df_tags.reset_index()
df_tags=df_tags.rename({'index':'id','TAGS':'text'}, axis=1)
df_tags

,id,text
0,0,maior vencimento
1,1,menor vencimento
2,2,vencimento básico
3,3,vencimento
4,4,remuneração
...,...,...
290,290,Nome servidor
291,291,função
292,292,nome do órgão
293,293,data início exercício


In [12]:
df_tags=df_tags.drop_duplicates()